In [41]:
import pandas as pd
import os
names = ['nb-asr-e5-1234.pickle', 'nb-asr-e10-1234.pickle','nb-asr-e40-1234.pickle','nb-asr-e40-1235.pickle', 'nb-asr-e40-1236.pickle', 'nb-asr-bench-jetson-nano-fp32.pickle', 'nb-asr-bench-gtx-1080ti-fp32.pickle']



# for name in names:
name = 'nb-asr-e10-1234.pickle'
nbasr_path = os.getcwd() + "/dataset/nbasr/" + name
with open(nbasr_path, 'rb') as f:
    header = pickle.load(f)
    data = pickle.load(f)
    print("\nFile: ", name)
    
    print(len(data))
    print(header)
    print(data[0])
    
def get_search_space(ops=None, nodes=None):
    ''' Return boundaries of the search space for the given list
        of available operations and number of nodes. 
    '''
    ops = ops if ops is not None else all_ops
    nodes = nodes if nodes is not None else default_nodes
    search_space = [[len(ops)] + [2]*(idx+1) for idx in range(nodes)]
    return search_space




File:  nb-asr-e10-1234.pickle
8242
{'version': 1, 'search_space': {'ops': ['linear', 'conv5', 'conv5d2', 'conv7', 'conv7d2', 'zero'], 'nodes': 3, 'shape': [[6, 2], [6, 2, 2], [6, 2, 2, 2]]}, 'seed': 1234, 'epochs': 10, 'dataset_type': 'training', 'columns': ['model_hash', 'val_per', 'test_per', 'arch_vec']}
['7b23d5980c2d91151af3dfcd98fce513', [0.6430896, 0.36368465, 0.30191937, 0.26333266, 0.2457993, 0.23856014, 0.23059042, 0.22819951, 0.22142525, 0.22335127], 0.23922383785247803, [(0, 0), (0, 0, 0), (0, 0, 0, 0)]]


In [43]:
print(header)
print(data[0])
print(header['search_space']['ops'])


{'version': 1, 'search_space': {'ops': ['linear', 'conv5', 'conv5d2', 'conv7', 'conv7d2', 'zero'], 'nodes': 3, 'shape': [[6, 2], [6, 2, 2], [6, 2, 2, 2]]}, 'seed': 1234, 'epochs': 10, 'dataset_type': 'training', 'columns': ['model_hash', 'val_per', 'test_per', 'arch_vec']}
['7b23d5980c2d91151af3dfcd98fce513', [0.6430896, 0.36368465, 0.30191937, 0.26333266, 0.2457993, 0.23856014, 0.23059042, 0.22819951, 0.22142525, 0.22335127], 0.23922383785247803, [(0, 0), (0, 0, 0), (0, 0, 0, 0)]]
['linear', 'conv5', 'conv5d2', 'conv7', 'conv7d2', 'zero']


In [32]:
ops = header['search_space']['ops']
nodes = header['search_space']['nodes']

get_search_space(ops, nodes)

[[6, 2], [6, 2, 2], [6, 2, 2, 2]]

In [37]:
import collections
def get_all_architectures(ops=None, nodes=None):
    ''' Yields all architecture configurations in the search space
    '''
    search_space = get_search_space(ops, nodes)
    flat = flatten(search_space)
    cfg = [0 for _ in range(len(flat))]
    end = False
    while not end:
        yield copy_structure(cfg, search_space)
        for dim in range(len(flat)):
            cfg[dim] += 1
            if cfg[dim] != flat[dim]:
                break
            cfg[dim] = 0
            if dim+1 >= len(flat):
                end = True
                
def copy_structure(data, shape):
    ''' Put data from ``data`` into nested containers like in ``shape``.
        This can be seen as "unflatten" operation, i.e.:
            seq == copy_structure(flatten(seq), seq)
    '''
    d_it = recursive_iter(data)

    def copy_level(s):
        if isinstance(s, collections.abc.Sequence):
            return type(s)(copy_level(ss) for ss in s)
        else:
            return next(d_it)
    return copy_level(shape)


# utils to work with nested collections
def recursive_iter(seq):
    ''' Iterate over elements in seq recursively (returns only non-sequences)
    '''
    if isinstance(seq, collections.abc.Sequence):
        for e in seq:
            for v in recursive_iter(e):
                yield v
    else:
        yield seq

In [38]:
get_all_architectures(ops, nodes)

<generator object get_all_architectures at 0x7fb94e27b040>

In [ ]:
import pandas as pd
import os

In [4]:
model = 'dscnn' # dscnn wav2vec
ft_extr = 'mfcc' #raw augmented mfcc

In [5]:
results_path = os.getcwd() + "/models/trained_models/" + model + "_" + ft_extr + "/results.csv"
df = pd.read_csv(results_path)
df

,epochs,batch_size,silence_percentage,unknown_percentage,validation_percentage,testing_percentage,background_frequency,background_volume,criterion,initial_lr,...,window_size_samples,feature_extraction_method,model_name,model_input_shape,test_acc,val_acc,run_test_sample_time,run_val_sample_time,time_get_train_batch,time_get_val_batch
0,40,-1,10.0,10.0,10.0,10.0,0,0,CrossEnt,0.001,...,640,mfcc,dscnn,"{49, 10}",91.901089,92.523131,0.007257,0.007565,0.844781,0.641427


In [6]:
models = ['dscnn', 'wav2vec', 'dscnn_maxpool', 'dscnn_avgpool', 'resnet']
ft_extr_methods = ['mfcc', 'raw', 'augmented', 'mel_spectrogram', 'dwt']

In [7]:
results_df = pd.DataFrame(columns=df.columns)
for model in models:
    for ft_extr in ft_extr_methods:
        results_path = os.getcwd() + "/models/trained_models/" + model + "_" + ft_extr + "/results.csv"
        try:
            df = pd.read_csv(results_path)
            results_df = results_df.append(df, ignore_index=True)
        except:
            pass

results_df

,epochs,batch_size,silence_percentage,unknown_percentage,validation_percentage,testing_percentage,background_frequency,background_volume,criterion,initial_lr,...,window_size_samples,feature_extraction_method,model_name,model_input_shape,test_acc,val_acc,run_test_sample_time,run_val_sample_time,time_get_train_batch,time_get_val_batch
0,40,-1,10.0,10.0,10.0,10.0,0,0,CrossEnt,0.001,...,640,mfcc,dscnn,"{49, 10}",91.901089,92.523131,0.007257,0.007565,0.844781,0.641427


In [9]:
model = 'wav2vec'
ft_extr = 'raw'
results_path = os.getcwd() + "/models/trained_models/" + model + "_" + ft_extr + "/results_batch256.csv"

In [10]:
results_path = os.getcwd() + "/models/trained_models/" + model + "_" + ft_extr + "/results_batch256.csv"
df = pd.read_csv(results_path)
results_df = results_df.append(df, ignore_index=True)

results_path = os.getcwd() + "/models/trained_models/" + model + "_" + ft_extr + "/results_batch512.csv"
df = pd.read_csv(results_path)
results_df = results_df.append(df, ignore_index=True)

results_df

,epochs,batch_size,silence_percentage,unknown_percentage,validation_percentage,testing_percentage,background_frequency,background_volume,criterion,initial_lr,...,window_size_samples,feature_extraction_method,model_name,model_input_shape,test_acc,val_acc,run_test_sample_time,run_val_sample_time,time_get_train_batch,time_get_val_batch
0,40,-1,10.0,10.0,10.0,10.0,0,0,CrossEnt,0.001,...,640,mfcc,dscnn,"{49, 10}",91.901089,92.523131,0.007257,0.007565,0.844781,0.641427
1,40,256,10.0,10.0,10.0,10.0,0,0,CrossEnt,0.001,...,640,raw,wav2vec,16000,96.484375,97.656250,0.003976,0.004615,0.022522,0.007729
2,40,512,10.0,10.0,10.0,10.0,0,0,CrossEnt,0.001,...,640,raw,wav2vec,16000,97.070312,96.484375,0.003427,0.003893,0.022816,0.007660
